# Filtering CAS Tables
[Getting Started with Python Integration to SAS® Viya® - Part 7 - Filtering CAS Tables](https://blogs.sas.com/content/sgf/2022/06/29/getting-started-with-python-integration-to-sas-viya-part-7-filtering-cas-tables/) blog post

## Import Packages
Visit the documentation for the SWAT [(SAS Scripting Wrapper for Analytics Transfer)](https://sassoftware.github.io/python-swat/index.html) package.

In [1]:
import swat
import pandas as pd

## custom personal module to connect to my CAS server environment
from casConnect import connect_to_cas 

## Make a Connection to CAS

##### To connect to the CAS server you will need:
1. the host name, 
2. the portnumber, 
3. your user name, and your password.

Visit the documentation [Getting Started with SAS® Viya® for Python](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg3/titlepage.htm) for more information about connecting to CAS.

**Be aware that connecting to the CAS server can be implemented in various ways, so you might need to see your system administrator about how to make a connection. Please follow company policy regarding authentication.**

In [2]:
##
## Connect to CAS
##

## General connection syntax
# conn = swat.CAS(host, port, username, password)

## Viya for Learners 3.5 connection
# hostValue = os.environ.get('CASHOST')
# portValue = os.environ.get('CASPORT')
# passwordToken=os.environ.get('SAS_VIYA_TOKEN')
# conn = swat.CAS(hostname=hostValue, port=portValue, password=passwordToken)

## Personal connection
try:
    conn = connect_to_cas()
    print('CAS connection succesful')
    print(conn)
except:
    print('No connection')
    pass

CAS connection succesful
CAS('ssemonthly.demo.sas.com', 443, protocol='https', name='py-session-1', session='8bf1e5b7-6378-3746-8257-b882140048dd')


## Load and explore data

In [3]:
tbl = conn.upload_file('https://raw.githubusercontent.com/sassoftware/sas-viya-programming/master/data/cars.csv',
                       casout={'name':'cars', 'caslib':'casuser'})

NOTE: Cloud Analytic Services made the uploaded file available as table CARS in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table CARS has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


In [4]:
display(type(tbl), tbl)

swat.cas.table.CASTable

CASTable('CARS', caslib='CASUSER(Peter.Styliadis@sas.com)')

In [5]:
tbl.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945.0,33337.0,3.5,6.0,265.0,17.0,23.0,4451.0,106.0,189.0
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820.0,21761.0,2.0,4.0,200.0,24.0,31.0,2778.0,101.0,172.0
2,Acura,TSX 4dr,Sedan,Asia,Front,26990.0,24647.0,2.4,4.0,200.0,22.0,29.0,3230.0,105.0,183.0
3,Acura,TL 4dr,Sedan,Asia,Front,33195.0,30299.0,3.2,6.0,270.0,20.0,28.0,3575.0,108.0,186.0
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755.0,39014.0,3.5,6.0,225.0,18.0,24.0,3880.0,115.0,197.0


In [6]:
tbl.shape

(428, 15)

## Filter a CAS Table

### Python Technique

In [7]:
makeFilter = (tbl['Make']=='Acura') | (tbl['Make']=='Toyota')
tbl[makeFilter].shape

(35, 15)

### Query Method

In [8]:
tbl.query("Make = 'Acura' or Make = 'Toyota'").shape

(35, 15)

### isin Method

In [9]:
tbl[tbl.Make.isin(['Acura', 'Toyota'])].shape

(35, 15)

### CAS Table WHERE Parameter

In [10]:
display(type(tbl), tbl)

swat.cas.table.CASTable

CASTable('CARS', caslib='CASUSER(Peter.Styliadis@sas.com)')

In [11]:
tbl.where = 'Make = "Acura" or Make = "Toyota"'
display(tbl)

CASTable('CARS', caslib='CASUSER(Peter.Styliadis@sas.com)', where='Make = "Acura" or Make = "Toyota"')

In [12]:
tbl.shape

(35, 15)

In [13]:
tbl.Make.value_counts()

Toyota    28
Acura      7
dtype: int64

#### Delete the WHERE parameter

In [14]:
tbl.del_params('where')
display(tbl)

CASTable('CARS', caslib='CASUSER(Peter.Styliadis@sas.com)')

In [15]:
tbl.shape

(428, 15)

## Terminate the CAS Connection

In [16]:
conn.terminate()